In [1]:
import numpy as np

In [6]:
idxs = {}
for i in range(6):
    idxs[i] = np.arange(i * 2688, (i + 1) * 2688 - 269)

In [7]:
idxs

{0: array([   0,    1,    2, ..., 2685, 2686, 2687]),
 1: array([2688, 2689, 2690, ..., 5373, 5374, 5375]),
 2: array([5376, 5377, 5378, ..., 8061, 8062, 8063]),
 3: array([ 8064,  8065,  8066, ..., 10749, 10750, 10751]),
 4: array([10752, 10753, 10754, ..., 13437, 13438, 13439]),
 5: array([13440, 13441, 13442, ..., 16125, 16126, 16127])}

In [8]:
2688 - 269

2419

In [36]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file, header=None)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Extract 11 input values and 1 target value
        inputs = self.data.iloc[idx, :-1].values.astype(np.float32)
        target = self.data.iloc[idx, -1].astype(np.float32)

        sample = {'inputs': inputs, 'target': target}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [40]:
import torch
import torch.nn as nn

class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x

In [45]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()

    for epoch in tqdm(range(num_epochs)):
        for batch in train_loader:
            inputs, target = batch['inputs'], batch['target']
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, target.unsqueeze(1))  # Ensure target has the right shape

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

In [47]:
def test_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in test_loader:
            inputs, target = batch['inputs'], batch['target']

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, target.unsqueeze(1))  # Ensure target has the right shape

            total_loss += loss.item()

    avg_loss = total_loss / len(test_loader)
    print(f'Test Loss: {avg_loss}')


In [61]:
from sklearn.model_selection import train_test_split
import torch.optim as optim

model = RegressionModel(11)
criterion = nn.MSELoss()

dataset = CustomDataset('./data/dataloader.csv')
batch_size = 32
print('splitting data')
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

print('loading in data')
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
print('loading in test data')
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

optimizer = optim.Adam(model.parameters(), lr=0.001)
# Train the model
print('start training')
train_model(model, train_loader, criterion, optimizer, num_epochs=10)

# Test the model
test_model(model, test_loader, criterion)

splitting data
loading in data
loading in test data
start training


 10%|█         | 1/10 [00:00<00:01,  5.48it/s]

Epoch 1/10, Loss: 61.65370178222656


 20%|██        | 2/10 [00:00<00:01,  5.51it/s]

Epoch 2/10, Loss: 34.128997802734375


...

Epoch 8/10, Loss: 47.76274108886719


 90%|█████████ | 9/10 [00:01<00:00,  5.18it/s]

Epoch 9/10, Loss: 41.91636657714844


100%|██████████| 10/10 [00:01<00:00,  5.30it/s]

Epoch 10/10, Loss: 38.62455749511719
Test Loss: 56.13319000395218


In [62]:
input_tensor = [32.59310344827586,32.59310344827586,89.0206896551724,67.1103448275862,508.96666666666664,462.58620689655174,1262.5217391304348,40.47543859649122,68.05172413793103,32.59310344827586,32.59310344827586]

input_tensor = torch.tensor(input_tensor, dtype=torch.float32)
model.eval()
with torch.no_grad():
    prediction = model(input_tensor)
predicted_value = prediction.item()

In [63]:
predicted_value

61.749935150146484

In [64]:
67.0 - predicted_value

5.250064849853516

In [65]:
input_tensor = [49.88793103448276,49.88793103448276,90.78793103448275,66.5844827586207,817.3315789473685,699.903448275862,1301.1739130434783,32.94736842105263,68.19137931034483,49.88793103448276,49.88793103448276]
input_tensor = torch.tensor(input_tensor, dtype=torch.float32)
model.eval()
with torch.no_grad():
    prediction = model(input_tensor)
predicted_value = prediction.item()